In [1]:
import pandas as pd
import matplotlib.pylab as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
frame_test = pd.read_csv("hq_markup_train.csv")
frame = pd.read_csv("markup_train.csv")

def get_sample(index, frame):
    uuid = frame.iloc[index, 0]
    with open(f'data/{uuid}', 'r') as f:
        content = f.read()
    rows = [line.split('\t') for line in content.strip().split('\n')]
    data = pd.DataFrame(rows, columns=['time', 'delta_p', 'p_'], dtype=float)
    return data


def find_empty_indexes_2(data):
    """
    Поиск id файлов из строк датафрейма для их исключения на этапе удаления данных из датафрейма
    """
    B=[]
    for i in range(data.shape[0]):
        try:
            df=get_sample(i,data)
        except ValueError:
            B.append(data.iloc[i,0])
    return B


# class SiamDataset(Dataset):
#     def __init__(self, siam_dataset_describe:pd.DataFrame):
#         super().__init__()
#         self.siam_dataset_describe = siam_dataset_describe

#     def __len__(self):
#         return self.siam_dataset_describe.shape[0]

#     def __getitem__(self, idx):
#         x = get_sample(idx, self.siam_dataset_describe) #.to_numpy(dtype=np.float64)
#         # t = x["time"].to_numpy(dtype=np.float64)
#         x = x[["delta_p", "p_"]].to_numpy(dtype=np.float64)
        


#         # 7) Возвращаем (X, Y)                                                  # Давление(атм) ([:, 1]) Давление(атм) ([:,1]) Давление(атм) ([:,1]) Давление(атм) ([:,1]) Давление(атм) ([:,1]) В какой момент? [:, 0]   В какой момент? [:, 0]
#         return x, self.siam_dataset_describe.iloc[idx][['Влияние ствола скважины_details', 'Радиальный режим_details', 'Линейный режим_details', 'Билинейный режим_details', 'Сферический режим_details', 'Граница постоянного давления_details', 'Граница непроницаемый разлом_details']].to_numpy(dtype=np.float32)

# B=find_empty_indexes_2(frame)#Поиск id с отсутствующими файлами для исключения
# frame=frame[~frame['file_name'].isin(B)]#Фильтрация данных
# frame = frame[~frame["file_name"].isin(frame_test["file_name"])]

In [8]:
import numpy as np
import pandas as pd
from scipy import stats
from itertools import groupby
from torch.utils.data import Dataset

def extract_features(data):
    """
    Extract binary features and their numerical coefficients from well test data.
    
    Args:
        data (pd.DataFrame): DataFrame with columns 'time', 'delta_p', 'p_'
    
    Returns:
        dict: Features with (presence, coefficient) tuples, e.g., {'bq': (1, None), 'wb': (0, None), ...}
    """
    # Initial quality checks
    if (len(data) < 10 or 
        any(data['p_'] <= 0) or 
        not all(np.diff(data['time']) > 0)):
        return {
            'bq': (1, None), 'wb': (0, None), 'ra': (0, None), 'li': (0, None),
            'bi': (0, None), 'sp': (0, None), 'pc': (0, None), 'im': (0, None)
        }

    # Prepare log-log data
    log_time = np.log10(data['time'].to_numpy())
    log_p_prime = np.log10(data['p_'].to_numpy())
    features = {}

    # Bad quality checks
    bq = 0
    dt = np.diff(data['time'])
    if max(dt) > 0.1 * (data['time'].iloc[-1] - data['time'].iloc[0]):  # Large gaps
        bq = 1
    dp = np.diff(data['p_'])
    if max(np.abs(dp)) > 5 * np.median(np.abs(dp)):  # Jumps in p_
        bq = 1
    delta_p = data['delta_p'].to_numpy()
    consecutive_same = max(sum(1 for _ in g) for k, g in groupby(delta_p))  # Stepwise delta_p
    if consecutive_same > 5:
        bq = 1
    sign_changes = np.sum(np.diff(np.sign(dp)) != 0)  # High noise
    if sign_changes > 0.5 * len(dp):
        bq = 1
    features['bq'] = (bq, None)

    # Define flow regimes and target slopes
    regimes = {'wb': 1.0, 'ra': 0.0, 'li': 0.5, 'bi': 0.25, 'sp': -0.5}

    # Function to find the best segment for a target slope

    def find_best_segment(log_time, log_p_prime, target_slope, tol=0.1, min_span=0.5):
        """
        Find the best segment in log-log data with a slope close to target_slope.
        
        Parameters:
        - log_time: Array of log10(time) values.
        - log_p_prime: Array of log10(p_) values.
        - target_slope: Expected slope for the regime (e.g., 0, 0.5).
        - tol: Slope tolerance (default 0.1).
        - min_span: Minimum span in log time (default 0.5 log cycles).
        
        Returns:
        - Tuple (start_idx, end_idx, slope, intercept) or None if no segment found.
        """
        n = len(log_time)
        
        # Precompute cumulative sums
        sum_x = np.cumsum(log_time)
        sum_y = np.cumsum(log_p_prime)
        sum_xx = np.cumsum(log_time**2)
        sum_xy = np.cumsum(log_time * log_p_prime)
        
        best_error = float('inf')
        best_segment = None
        
        # For each starting index i
        for i in range(n):
            # Find the smallest j where log_time[j] - log_time[i] >= min_span
            j_start = np.searchsorted(log_time, log_time[i] + min_span, side='left')
            
            # Check all j from j_start to n-1
            for j in range(max(j_start, i + 1), n):
                m = j - i + 1
                
                # Compute sums for segment i to j using cumulative sums
                sx = sum_x[j] - (sum_x[i - 1] if i > 0 else 0)
                sy = sum_y[j] - (sum_y[i - 1] if i > 0 else 0)
                sxx = sum_xx[j] - (sum_xx[i - 1] if i > 0 else 0)
                sxy = sum_xy[j] - (sum_xy[i - 1] if i > 0 else 0)
                
                # Compute slope k
                denom = m * sxx - sx**2
                if denom == 0:  # Avoid division by zero
                    continue
                
                k = (m * sxy - sx * sy) / denom
                
                # Check if slope is within tolerance
                if abs(k - target_slope) < tol:
                    error = abs(k - target_slope)
                    if error < best_error:
                        best_error = error
                        intercept = (sy - k * sx) / m
                        best_segment = (i, j, k, intercept)
        
        return best_segment

    # Wellbore storage (wb): must start at the beginning
    i = 0
    j_max = len(log_time) - 1
    while j_max > i and log_time[j_max] - log_time[i] >= 0.5:
        x = log_time[i:j_max + 1]
        y = log_p_prime[i:j_max + 1]
        slope, intercept, _, _, _ = stats.linregress(x, y)
        if abs(slope - 1.0) < 0.1:
            features['wb'] = (1, intercept)
            break
        j_max -= 1
    else:
        features['wb'] = (0, None)

    # Other flow regimes
    for regime, target_slope in regimes.items():
        if regime == 'wb':
            continue
        segment = find_best_segment(log_time, log_p_prime, target_slope)
        if segment:
            _, _, _, intercept = segment
            features[regime] = (1, intercept)
        else:
            features[regime] = (0, None)

    # Boundaries (pc and im): analyze last 0.5 log cycles
    idx_last = np.searchsorted(log_time, log_time[-1] - 0.5, side='right')
    if idx_last < len(log_time):
        x_last = log_time[idx_last:]
        y_last = log_p_prime[idx_last:]
        if len(x_last) > 1:
            slope_last, _, _, _, _ = stats.linregress(x_last, y_last)
            if slope_last < -0.5:
                features['pc'] = (1, data['time'].iloc[idx_last])
            else:
                features['pc'] = (0, None)
            if slope_last > 0.5:
                features['im'] = (1, data['time'].iloc[idx_last])
            else:
                features['im'] = (0, None)
        else:
            features['pc'] = (0, None)
            features['im'] = (0, None)
    else:
        features['pc'] = (0, None)
        features['im'] = (0, None)

    return features

# Enhanced SiamDataset class
class SiamDataset(Dataset):
    def __init__(self, siam_dataset_describe: pd.DataFrame):
        super().__init__()
        self.siam_dataset_describe = siam_dataset_describe

    def __len__(self):
        return self.siam_dataset_describe.shape[0]

    def __getitem__(self, idx):
        # Get raw data
        data = get_sample(idx, self.siam_dataset_describe)
        # Extract features
        features = extract_features(data)
        return features

# Provided function (assumed available)
def get_sample(index, frame):
    uuid = frame.iloc[index, 0]
    with open(f'data/{uuid}', 'r') as f:
        content = f.read()
    rows = [line.split('\t') for line in content.strip().split('\n')]
    data = pd.DataFrame(rows, columns=['time', 'delta_p', 'p_'], dtype=float)
    return data

i = 0 
dataset_lol = SiamDataset(frame_test)
for feature in dataset_lol:
    print(feature)
    i+= 1
    if i>10:
        break

{'bq': (1, None), 'wb': (1, np.float64(2.0283219859137147)), 'ra': (1, np.float64(0.10545369102160654)), 'li': (1, np.float64(-0.20842201959350842)), 'bi': (1, np.float64(0.02285986620647338)), 'sp': (1, np.float64(0.047322571856873163)), 'pc': (0, None), 'im': (1, np.float64(4.050000000279397))}
{'bq': (1, None), 'wb': (0, None), 'ra': (1, np.float64(1.1838722595163604)), 'li': (1, np.float64(1.3519600807905274)), 'bi': (1, np.float64(1.2489142870628902)), 'sp': (1, np.float64(0.16104224414638724)), 'pc': (0, None), 'im': (0, None)}
{'bq': (1, None), 'wb': (0, None), 'ra': (1, np.float64(0.5715950993667769)), 'li': (1, np.float64(0.6023708424326463)), 'bi': (1, np.float64(-0.9253292386490699)), 'sp': (1, np.float64(0.6287709937329575)), 'pc': (0, None), 'im': (1, np.float64(51.0))}
{'bq': (1, None), 'wb': (0, None), 'ra': (1, np.float64(1.6614456549529717)), 'li': (1, np.float64(1.8170324414889205)), 'bi': (1, np.float64(1.5663522010919986)), 'sp': (1, np.float64(2.3372663679346495)),

In [42]:
import numpy as np
from scipy.signal import savgol_filter




def find_best_segment(log_time, log_p_prime, target_slope, tol=0.2, min_span=0.3):
    """
    Find the best segment in log-log data with a slope close to target_slope.
    
    Parameters:
    - log_time: Array of log10(time) values.
    - log_p_prime: Array of log10(p_) values.
    - target_slope: Expected slope (e.g., 0 for radial, 0.5 for linear).
    - tol: Slope tolerance (default 0.2).
    - min_span: Minimum span in log time (default 0.3 log cycles).
    
    Returns:
    - Tuple (start_idx, end_idx, slope, intercept) or None if no segment found.
    """
    n = len(log_time)
    
    # Smooth the pressure derivative to reduce noise
    log_p_prime_smooth = savgol_filter(log_p_prime, window_length=3, polyorder=2)
    
    # Precompute cumulative sums for efficiency
    sum_x = np.cumsum(log_time)
    sum_y = np.cumsum(log_p_prime_smooth)
    sum_xx = np.cumsum(log_time**2)
    sum_xy = np.cumsum(log_time * log_p_prime_smooth)
    
    best_error = float('inf')
    best_segment = None
    
    # Loop over starting indices
    for i in range(n):
        # Find the earliest j where segment spans at least min_span
        j_start = np.searchsorted(log_time, log_time[i] + min_span, side='left')
        
        # Check all valid end indices
        for j in range(max(j_start, i + 1), n):
            m = j - i + 1
            
            # Compute segment sums using cumulative sums
            sx = sum_x[j] - (sum_x[i - 1] if i > 0 else 0)
            sy = sum_y[j] - (sum_y[i - 1] if i > 0 else 0)
            sxx = sum_xx[j] - (sum_xx[i - 1] if i > 0 else 0)
            sxy = sum_xy[j] - (sum_xy[i - 1] if i > 0 else 0)
            
            # Calculate slope (k)
            denom = m * sxx - sx**2
            if denom == 0:  # Avoid division by zero
                continue
            
            k = (m * sxy - sx * sy) / denom
            
            # Check if slope matches target within tolerance
            if abs(k - target_slope) < tol:
                error = abs(k - target_slope)
                if error < best_error:
                    best_error = error
                    intercept = (sy - k * sx) / m
                    best_segment = (i, j, k, intercept)
    
    return best_segment


def analyze_well_test_data(time, p_):
    """
    Analyze well test data to identify flow regimes and boundaries.
    
    Parameters:
    - time: Array of time values.
    - p_: Array of pressure derivative values.
    
    Returns:
    - Dictionary with feature flags and coefficients.
    """
    # Data preparation
    log_time = np.log10(time)
    log_p_prime = np.log10(p_)
    # print("We are in")
    
    # Initialize output dictionary
    features = {
        'bq': 0,  # Bad quality
        'wb': 0,  # Wellbore storage
        'ra': 0,  # Radial flow
        'li': 0,  # Linear flow
        'bi': 0,  # Bilinear flow
        'sp': 0,  # Spherical flow
        'pc': 0,  # Constant pressure boundary
        'im': 0,  # Impermeable boundary
        'coeffs': {}  # Coefficients for detected features
    }
    
    # Bad quality checks (simplified example)
    if (len(time) < 5 or np.any(p_ <= 0) or np.any(np.diff(time) <= 0) or 
        np.max(np.abs(np.diff(p_))) > 10 * np.median(np.abs(np.diff(p_)))):
        features['bq'] = 1
        # return features
    
    # Wellbore storage (slope ≈ 1 at start)
    # for slop_p in (-10, 10):   + slop_p/100
    wb_segment = find_best_segment(log_time[:log_time.shape[0]//2], log_p_prime[:log_time.shape[0]//2], target_slope= 1 )
    if wb_segment:
        features['wb'] = 1
        features['coeffs']['wb'] = {'slope': wb_segment[2], 'intercept': wb_segment[3]}
    
    # Flow regimes
    flow_regimes = {
        'ra': 0,    # Radial flow
        'li': 0.5,  # Linear flow
        'bi': 0.25, # Bilinear flow
        'sp': -0.5  # Spherical flow
    }
    for regime, slope in flow_regimes.items():
        segment = find_best_segment(log_time, log_p_prime, target_slope=slope)
        if segment:
            features[regime] = 1
            features['coeffs'][regime] = {'slope': segment[2], 'intercept': segment[3]}
    
    # Boundary detection (last 0.5 log cycles)
    last_span = log_time[-1] - 0.5
    mask = log_time >= last_span
    log_time_last = log_time[mask]
    log_p_prime_last = log_p_prime[mask]
    
    if len(log_time_last) > 1:
        # Fit a line to the last segment
        coeffs = np.polyfit(log_time_last, log_p_prime_last, 1)
        slope = coeffs[0]
        
        if slope < -0.5:  # Decreasing derivative
            features['pc'] = 1
            features['coeffs']['pc'] = {'slope': slope, 'intercept': coeffs[1]}
        elif slope > 0.5:  # Increasing derivative
            features['im'] = 1
            features['coeffs']['im'] = {'slope': slope, 'intercept': coeffs[1]}
    
    return features




# Provided function (assumed available)
def get_sample(index, frame):
    uuid = frame.iloc[index, 0]
    with open(f'data/{uuid}', 'r') as f:
        content = f.read()
    rows = [line.split('\t') for line in content.strip().split('\n')]
    data = pd.DataFrame(rows, columns=['time', 'delta_p', 'p_']).to_numpy(np.float32)
    return data


# Enhanced SiamDataset class
class SiamDataset(Dataset):
    def __init__(self, siam_dataset_describe: pd.DataFrame):
        super().__init__()
        self.siam_dataset_describe = siam_dataset_describe

    def __len__(self):
        return self.siam_dataset_describe.shape[0]

    def __getitem__(self, idx):
        # Get raw data
        data = get_sample(idx, self.siam_dataset_describe)
        # Extract features
        # print("I'm here")
        features = analyze_well_test_data(data[:,0], data[:,2])
        # print("Fuck")
        return features



# Example usage
# time = np.logspace(0, 3, 100)  # Sample time data
# p_ = np.concatenate([time[:20], 10 * np.ones(80)])  # Sample p_ with wellbore storage and radial flow

i = 0 

dataset_lol = SiamDataset(frame_test)
for feature in dataset_lol:
    # print(feature["coeffs"])
    features_names = {
        'wb': 0,  # Wellbore storage
        'ra': 0,  # Radial flow
        'li': 0,  # Linear flow
        'bi': 0,  # Bilinear flow
        'sp': 0,  # Spherical flow
        'pc': 0,  # Constant pressure boundary
        'im': 0,  # Impermeable boundary
    }

    
    a = [feature["coeffs"].get(name, None) for name in features_names]
    print([item.get("intercept") if item != None else None for item in a ])

        
    i += 1
    if i > 10:
        break

[np.float32(2.0283384), np.float32(0.10545349), np.float32(-0.20842344), np.float32(0.022859583), np.float32(0.047323275), None, np.float64(-0.39886000020478085)]
[None, np.float32(1.1838709), np.float32(1.3545445), np.float32(1.2489133), np.float32(0.2759524), None, None]
[np.float32(-0.1996168), np.float32(0.57159466), np.float32(0.6023708), np.float32(-0.92535007), np.float32(0.628771), None, np.float64(-6.349284544894653)]
[np.float32(1.9778912), np.float32(1.5530326), np.float32(1.8170322), np.float32(1.5663522), np.float32(2.337265), None, None]
[np.float32(1.2263768), np.float32(1.1451756), np.float32(0.34229764), np.float32(0.68409145), np.float32(-1.437851), None, None]
[None, None, None, None, np.float32(1.3498039), None, np.float64(-5.730709149234244)]
[None, np.float32(1.1189736), None, np.float32(1.5080851), None, None, None]
[np.float32(1.9484775), np.float32(1.1269311), np.float32(1.1346841), np.float32(1.032888), np.float32(1.8129542), np.float64(2.693913972142928), Non

In [5]:
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import math

# ----- Ваши исходные классы и функции -----

def get_sample(index, frame):
    """
    Чтение файла с данными по его uuid (frame.iloc[index, 0])
    и парсинг в DataFrame со столбцами ['time', 'delta_p', 'p_'].
    """
    uuid = frame.iloc[index, 0]
    with open(f'data/{uuid}', 'r') as f:
        content = f.read()
    rows = [line.split('\t') for line in content.strip().split('\n')]
    data = pd.DataFrame(rows, columns=['time', 'delta_p', 'p_'], dtype=float)
    return data

class SiamDataset(Dataset):
    def __init__(self, siam_dataset_describe: pd.DataFrame):
        super().__init__()
        self.siam_dataset_describe = siam_dataset_describe

    def __len__(self):
        return self.siam_dataset_describe.shape[0]

    def __getitem__(self, idx):
        x = get_sample(idx, self.siam_dataset_describe)
        # Здесь можно вернуть только нужные столбцы или все
        x = x[["delta_p", "p_"]].to_numpy(dtype=np.float64)
        return x


# ----- Основная логика определения признаков -----

def detect_flow_regimes(data: pd.DataFrame, 
                        min_log_cycle=0.5,
                        slope_tolerance=0.1) -> dict:
    """
    Определяет 8 бинарных признаков:
      bq, wb, ra, li, bi, sp, pc, im
    и возвращает также численные коэффициенты:
      - wb, ra, li, bi, sp -> коэффициент b (если признак обнаружен)
      - pc, im -> время начала эффекта (если признак обнаружен)

    Параметры:
      data: DataFrame c колонками ["time", "delta_p", "p_"].
      min_log_cycle: минимальная «длина» участка на лог-шкале,
                     чтобы считать режим достоверным (0.5 цикла = фактор ~3.16 по времени).
      slope_tolerance: допуск на ошибку в определении теоретического наклона
                       (зависит от качества данных, обычно 0.05–0.15).

    Возвращает словарь вида:
      {
        'bq': int (0 или 1),
        'wb': (int, float or None),  # (0 или 1, b)
        'ra': (int, float or None),
        'li': (int, float or None),
        'bi': (int, float or None),
        'sp': (int, float or None),
        'pc': (int, float or None),  # (0 или 1, time)
        'im': (int, float or None)   # (0 или 1, time)
      }
    """

    # Инициализация результирующих значений
    results = {
        'bq': 0,
        'wb': (0, None),
        'ra': (0, None),
        'li': (0, None),
        'bi': (0, None),
        'sp': (0, None),
        'pc': (0, None),
        'im': (0, None)
    }

    # 1. Проверка «качества» данных (bq).
    #    Ниже простой пример проверки: ищем «аномальные» скачки и
    #    смотрим, хватает ли данных для анализа.
    if len(data) < 5:
        # Если данных слишком мало, ставим bq = 1
        results['bq'] = 1
        return results
    
    # Простейшая проверка на шум по производной:
    delta_p_values = data["delta_p"].values
    # Можем измерить относительные скачки
    diffs = np.abs(np.diff(delta_p_values))
    mean_val = np.mean(delta_p_values) if np.mean(delta_p_values) != 0 else 1e-9
    if np.max(diffs) / np.abs(mean_val) > 10:  # условный порог
        results['bq'] = 1
    # Аналогично можно прописать любые другие евристики (пропуски, «ступени» и т. д.)

    # 2. Логарифмическая подготовка данных
    #    Удаляем строки, где time <= 0 или p_ <= 0, иначе log() не определён.
    clean_data = data[(data["time"] > 0) & (data["p_"] > 0)].copy()
    clean_data["log_time"] = np.log10(clean_data["time"])
    clean_data["log_p_der"] = np.log10(clean_data["p_"])

    if len(clean_data) < 5:
        # После фильтрации слишком мало точек
        return results

    # Для упрощения будем искать основные режимы на всём диапазоне,
    # рассматривая «скользящие» интервалы.
    # В реальности лучше искать отдельные непрерывные сегменты, исключать шум и т.д.
    log_times = clean_data["log_time"].values
    log_p_der = clean_data["log_p_der"].values
    times = clean_data["time"].values

    # 3. Поиск характерных режимов по наклону (wb, ra, li, bi, sp).
    #    Ищем в середине/начале данных, исключая «конец» (там могут быть границы).
    #    Для упрощения возьмём квазиподход: один общий лин. регресс на ~первой половине
    #    или на выделенном «оконном» участке. В реальных задачах делается разбиение
    #    на отдельные интервалы и анализ наклона на каждом.

    # Пример (очень упрощённый): возьмём первые 50% точек, сделаем регрессию
    half_index = len(clean_data) // 2
    # Можно выбрать ранние времена для wb, sp (они "ранние режимы"),
    # а для ra, li, bi – более середину графика. Ниже – просто пример.

    # Функция для вычисления наклона и b через МНК на log-log
    def linear_regression_log(x, y):
        # x, y — в логарифмическом масштабе
        # решим min || (k*x + b) - y ||^2
        # k = cov(x,y)/var(x), b = mean(y) - k*mean(x)
        x_mean = np.mean(x)
        y_mean = np.mean(y)
        cov_xy = np.mean(x*y) - x_mean*y_mean
        var_x = np.mean(x*x) - x_mean*x_mean
        if abs(var_x) < 1e-12:
            return None, None  # Мало ли
        k = cov_xy / var_x
        b = y_mean - k*x_mean
        return k, b

    # --- (A) Проба детекции wb (wellbore storage) ---
    # Ищем в самом начале (возьмём, к примеру, 20% самых ранних точек)
    n_start = max(3, len(clean_data)//5)
    early_log_t = log_times[:n_start]
    early_log_p = log_p_der[:n_start]
    k_early, b_early = linear_regression_log(early_log_t, early_log_p)
    # Если k_early в пределах 1±slope_tolerance — считаем wb проявленным
    if k_early is not None and abs(k_early - 1.0) < slope_tolerance:
        results['wb'] = (1, b_early)

    # --- (B) Проба детекции sp (spherical, k=-0.5) ---
    # Допустим, тоже в начале, но сдвинемся чуть дальше wb.
    # Тут – исключительно пример; лучше искать в явном отдельном участке.
    sp_index_start = n_start
    sp_index_end = sp_index_start + (len(clean_data)//5)
    if sp_index_end <= len(clean_data):
        log_t_sp = log_times[sp_index_start:sp_index_end]
        log_p_sp = log_p_der[sp_index_start:sp_index_end]
        k_sp, b_sp = linear_regression_log(log_t_sp, log_p_sp)
        if k_sp is not None and abs(k_sp + 0.5) < slope_tolerance:
            # Проверка, что заняло >= 0.5 логарифмического цикла
            t_min, t_max = times[sp_index_start], times[sp_index_end-1]
            if math.log10(t_max) - math.log10(t_min) >= min_log_cycle:
                results['sp'] = (1, b_sp)

    # --- (C) Проба детекции ra (radial, k=0), li (k=0.5), bi (k=0.25) ---
    # Для упрощения возьмём «среднюю» часть графика
    mid_index_start = len(clean_data)//3
    mid_index_end = (2*len(clean_data))//3
    log_t_mid = log_times[mid_index_start:mid_index_end]
    log_p_mid = log_p_der[mid_index_start:mid_index_end]
    k_mid, b_mid = linear_regression_log(log_t_mid, log_p_mid)

    if k_mid is not None:
        # Проверяем ближайший «теоретический» режим
        # Можно искать, к какому из {0, 0.25, 0.5} ближе всего
        candidates = {
            'ra': 0.0,
            'bi': 0.25,
            'li': 0.5
        }
        best_mode = None
        best_diff = 999
        for mode_name, k_theory in candidates.items():
            diff = abs(k_mid - k_theory)
            if diff < best_diff:
                best_diff = diff
                best_mode = mode_name
        # Если разница не превышает slope_tolerance, объявляем этот режим
        if best_diff < slope_tolerance:
            # Дополнительно проверяем длину по времени
            t_min, t_max = times[mid_index_start], times[mid_index_end-1]
            if math.log10(t_max) - math.log10(t_min) >= min_log_cycle:
                results[best_mode] = (1, b_mid)

    # 4. Поиск граничных эффектов pc (constant pressure boundary) и im (impermeable boundary).
    #    Анализируем «конец» кривой (последние 20-30% точек).
    tail_index_start = (4*len(clean_data))//5
    log_t_tail = log_times[tail_index_start:]
    log_p_tail = log_p_der[tail_index_start:]
    t_tail = times[tail_index_start:]

    # Проверим поведение последнего участка:
    # - Для pc: p' падает к 0 (на log-log это может выглядеть как «ухудшение» корреляции)
    # - Для im: p' растёт, старательно приближаясь к наклону 1, но может не успеть.
    # Упростим логику: возьмём лин. регрессию на хвосте:
    k_tail, b_tail = linear_regression_log(log_t_tail, log_p_tail)
    if k_tail is not None:
        # Сравниваем: если k_tail < -0.2 (например), то возможен pc
        # Если k_tail > 0.7 (например) — возможен im
        # Это очень условные значения, требуют уточнения.

        # Определим время начала эффекта как первую точку «хвоста».
        start_time_boundary = t_tail[0]

        if k_tail < -0.2:
            # Допустим, признаём pc
            results['pc'] = (1, start_time_boundary)

        elif k_tail > 0.7:
            # Допустим, признаём im
            results['im'] = (1, start_time_boundary)

    return results

# ----- Пример использования -----

def process_entire_dataset(siam_dataset_describe: pd.DataFrame):
    """
    Пример функции, которая проходит по всему DataFrame-описанию
    (где хранятся UUID файлов), читает каждый файл, анализирует
    и возвращает результат в виде таблицы (DataFrame).
    """
    results_list = []

    for idx in range(len(siam_dataset_describe)):
        # Считываем данные из файла
        df = get_sample(idx, siam_dataset_describe)

        # Определяем признаки
        regime_info = detect_flow_regimes(df)

        # Сохраним в удобном формате
        uuid = siam_dataset_describe.iloc[idx, 0]
        row_result = {
            'uuid': uuid,
            # Бинарные флаги
            'bq': regime_info['bq'],
            'wb': regime_info['wb'][0],
            'ra': regime_info['ra'][0],
            'li': regime_info['li'][0],
            'bi': regime_info['bi'][0],
            'sp': regime_info['sp'][0],
            'pc': regime_info['pc'][0],
            'im': regime_info['im'][0],
            # Численные коэффициенты
            'wb_b': regime_info['wb'][1],
            'ra_b': regime_info['ra'][1],
            'li_b': regime_info['li'][1],
            'bi_b': regime_info['bi'][1],
            'sp_b': regime_info['sp'][1],
            'pc_start_time': regime_info['pc'][1],
            'im_start_time': regime_info['im'][1]
        }
        results_list.append(row_result)

    # Формируем DataFrame с результатами
    results_df = pd.DataFrame(results_list)
    return results_df


if __name__ == "__main__":
    # Пример: у вас есть DataFrame siam_dataset_describe
    # c колонкой, например, "uuid" (или чем-то аналогичным).
    # Ниже - условный пример. В реальности вы уже имеете этот DataFrame.
    
    # Заглушка: названия файлов
    frame_test

    # Вызываем обработку
    results = process_entire_dataset(frame_test)
    print(results)


                                     uuid  bq  wb  ra  li  bi  sp  pc  im  \
0    e9c7e07f-b723-4da3-918e-7dbcb360c830   0   0   0   0   0   0   0   1   
1    726565c9-e05b-4a28-9079-22d94f9bb9e2   0   0   1   0   0   0   0   0   
2    c2a4d43d-5a93-4c37-81b6-d4fdc5b0280b   0   0   0   0   0   0   1   0   
3    15e04219-9fa0-401d-8b42-833b71ccafb3   0   0   0   0   0   0   0   0   
4    d43f459d-97fa-4ab0-bacc-43e2cb73e1b9   0   0   0   1   0   0   0   0   
..                                    ...  ..  ..  ..  ..  ..  ..  ..  ..   
495  a0448188-be5d-4ad6-929f-1870710e224b   0   0   0   0   0   0   1   0   
496  c295e7be-009c-4893-94c0-194d449077ed   0   0   1   0   0   0   1   0   
497  ac06114e-e466-497a-a0b7-5166d784140e   0   0   0   0   1   0   0   1   
498  0fd9d92c-a6d6-44c2-80b2-7f3cfcbea3da   0   0   0   0   0   0   1   0   
499  b145d37a-2b67-47e1-b4d4-a70a59192187   0   0   0   1   0   0   0   0   

     wb_b      ra_b      li_b      bi_b  sp_b  pc_start_time  im_start_time

In [19]:
import numpy as np
import pandas as pd

import math

from torch.utils.data import Dataset, DataLoader

# Для сглаживания
from scipy.signal import savgol_filter

# Для робастной регрессии
from sklearn.linear_model import RANSACRegressor
from sklearn.linear_model import LinearRegression


# ==== Ваши исходные классы и функции ====

def get_sample(index, frame):
    """
    Чтение файла с данными по его uuid (frame.iloc[index, 0])
    и парсинг в DataFrame со столбцами ['time', 'delta_p', 'p_'].
    """
    uuid = frame.iloc[index, 0]
    with open(f'data/{uuid}', 'r') as f:
        content = f.read()
    rows = [line.split('\t') for line in content.strip().split('\n')]
    data = pd.DataFrame(rows, columns=['time', 'delta_p', 'p_'], dtype=float)
    return data

class SiamDataset(Dataset):
    def __init__(self, siam_dataset_describe: pd.DataFrame):
        super().__init__()
        self.siam_dataset_describe = siam_dataset_describe

    def __len__(self):
        return self.siam_dataset_describe.shape[0]

    def __getitem__(self, idx):
        x = get_sample(idx, self.siam_dataset_describe)
        x = x[["delta_p", "p_"]].to_numpy(dtype=np.float64)
        return x


# ============ ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ ============

def check_data_quality(df, max_rel_jump=10.0, min_points=5):
    """
    Простая эвристическая проверка «качества» данных:
      - достаточно ли точек?
      - нет ли слишком больших скачков delta_p?
    Возвращает:
      bq_flag (int) — 1 если данные «плохие», иначе 0
    """
    if len(df) < min_points:
        return 1

    delta_p_values = df["delta_p"].values
    mean_val = np.mean(delta_p_values) if np.mean(delta_p_values) != 0 else 1e-9
    if mean_val == 0:
        return 1

    diffs = np.abs(np.diff(delta_p_values))
    max_jump = np.max(diffs)
    rel_jump = max_jump / abs(mean_val)
    if rel_jump > max_rel_jump:
        return 1

    return 0


def to_log_scale(df):
    """
    Переходит в логарифмические координаты:
      log_time = log10(time)
      log_p_der = log10(p_)
    Удаляет строки, где time <= 0 или p_ <= 0.
    Возвращает очищенный DataFrame с новыми столбцами.
    """
    df = df.copy()
    df = df[(df["time"] > 0) & (df["p_"] > 0)].sort_values("time")
    df["log_time"] = np.log10(df["time"])
    df["log_p_der"] = np.log10(df["p_"])
    return df


def savgol_smooth(series, window_size=5, polyorder=2):
    """
    Пример сглаживания серии методом Савицкого–Голея (savgol_filter).
    Перед фильтрацией желательно убедиться,
    что window_size <= длина массива.
    """
    if len(series) < window_size:
        return series
    return savgol_filter(series, window_length=window_size, polyorder=polyorder)


def robust_regression_log(x, y):
    """
    Проводит робастную линейную регрессию (RANSAC) на данных в лог-лог координатах.
    x, y — numpy-массивы (одномерные), уже в логарифмической форме.
    Возвращает (k, b) или (None, None), если регрессия не удалась.
    Уравнение: y ~ k*x + b
    """
    if len(x) < 2:
        return None, None
    model = RANSACRegressor( 
                            min_samples=max(2, len(x)), 
                            loss = "absolute_error",
                            residual_threshold=0.1,  # Порог чувствительности (подбирается)
                            max_trials=200)
    X_reshape = x.reshape(-1, 1)
    try:
        model.fit(X_reshape, y)
        k = model.estimator_.coef_[0]
        b = model.estimator_.intercept_
        return k, b
    except:
        return None, None


def segment_log_curve(log_t, log_p, min_log_cycle=0.3, step_fraction=0.1):
    """
    Пример «скользящей» сегментации:
      - идём по данным от начала к концу,
      - «накапливаем» точки, пока
         (log10(t_max) - log10(t_min)) < min_log_cycle
      - как только длина >= min_log_cycle, делаем робастную регрессию
      - затем двигаемся далее с шагом (например step_fraction от текущего окна)
    
    Возвращает список сегментов:
      [ (start_index, end_index, k, b), ... ]
    Обратите внимание, что это упрощённый вариант. 
    """

    n = len(log_t)
    segments = []
    i = 0
    while i < n-1:
        # Начинаем новый сегмент
        j = i
        t_min = 10**(log_t[i])
        while j < n-1:
            t_max = 10**(log_t[j])
            # Проверяем, сколько логарифмических циклов набрали
            if math.log10(t_max) - math.log10(t_min) >= min_log_cycle:
                # Делаем регрессию на [i..j]
                segment_x = log_t[i:j+1]
                segment_y = log_p[i:j+1]

                k_seg, b_seg = robust_regression_log(segment_x, segment_y)
                if k_seg is not None:
                    segments.append((i, j, k_seg, b_seg))

                # Сдвигаем i «шагом»
                step = max(1, int((j - i + 1)*step_fraction))
                i = j + step
                break
            j += 1

        else:
            # Если цикл while j < n-1 завершился без break,
            # значит мы дошли до конца без достаточной длины
            break

    return segments


def classify_segment(k, slope_tolerance=0.1):
    """
    Сопоставляет найденный наклон k с теоретическим значением
    среди wb (1), sp (-0.5), ra (0), li (0.5), bi (0.25).
    Возвращает метку из {wb, sp, ra, li, bi} или None.
    """
    modes = {
        "wb": 1.0,
        "sp": -0.5,
        "ra": 0.0,
        "bi": 0.25,
        "li": 0.5
    }
    best_mode = None
    best_diff = float('inf')
    for m, k_theor in modes.items():
        diff = abs(k - k_theor)
        if diff < best_diff:
            best_diff = diff
            best_mode = m

    if best_diff <= slope_tolerance:
        return best_mode
    else:
        return None


def detect_boundaries(log_t, log_p, time_arr,
                      slope_tolerance=0.3,
                      boundary_fraction=0.25):
    """
    Упрощённый метод детекции граничных эффектов pc/im на «хвосте»:
      - Берём последние boundary_fraction (например 25%) точек
      - Проводим робастную регрессию
      - Смотрим, куда «тянет» наклон.
      - Возвращаем (pc_flag, pc_time, im_flag, im_time).
    """
    n = len(log_t)
    start_idx = int(n * (1 - boundary_fraction))
    if start_idx < 0:
        start_idx = 0

    X_tail = log_t[start_idx:]
    Y_tail = log_p[start_idx:]
    k_tail, b_tail = robust_regression_log(X_tail, Y_tail)
    pc_flag = 0
    pc_time = None
    im_flag = 0
    im_time = None

    if k_tail is not None:
        # Условные пороги для pc и im:
        # pc → k_tail < -0.2 или стремится к 0
        # im → k_tail > 0.7 или стремится к 1
        # Эти пороги зависят от реальных данных
        time_boundary = time_arr[start_idx]  # момент начала хвоста

        if k_tail < -0.2:
            pc_flag = 1
            pc_time = time_boundary
        elif k_tail > 0.7:
            im_flag = 1
            im_time = time_boundary

    return pc_flag, pc_time, im_flag, im_time


# ============ ГЛАВНАЯ ФУНКЦИЯ ОПРЕДЕЛЕНИЯ ПРИЗНАКОВ ============

def detect_flow_regimes_advanced(data: pd.DataFrame,
                                 min_log_cycle=0.5,
                                 slope_tolerance=0.1) -> dict:
    """
    Более продвинутая версия детекции 8 признаков:
      bq, wb, ra, li, bi, sp, pc, im
    с использованием:
      - предварительного сглаживания
      - робастной регрессии (RANSAC)
      - сегментации методом «скользящих окон»
      - уточнённой логики для pc/im
    Возвращает словарь с бинарными признаками и коэффициентами:
      {
        'bq': int,
        'wb': (0 или 1, b_wb),
        'ra': (0 или 1, b_ra),
        'li': (0 или 1, b_li),
        'bi': (0 или 1, b_bi),
        'sp': (0 или 1, b_sp),
        'pc': (0 или 1, t_pc),
        'im': (0 или 1, t_im)
      }
    """

    # Инициализация выхода
    results = {
        'bq': 0,
        'wb': (0, None),
        'ra': (0, None),
        'li': (0, None),
        'bi': (0, None),
        'sp': (0, None),
        'pc': (0, None),
        'im': (0, None)
    }

    # 1) Проверка и первичная очистка
    bq_flag = check_data_quality(data)
    results['bq'] = bq_flag

    # Если «некачественные» данные, формально можно всё равно попробовать
    # определить режимы — бывает, что часть данных всё же годная.
    # Но в реальных условиях часто просто прерываемся.

    # 2) Логарифмический DataFrame
    df_log = to_log_scale(data)
    if len(df_log) < 10:
        return results

    # 3) Сортируем по времени и сглаживаем p'
    df_log = df_log.sort_values("log_time").reset_index(drop=True)
    df_log["log_p_der_smooth"] = savgol_smooth(df_log["log_p_der"].values, 
                                               window_size=5, polyorder=2)                         # ИЗМЕНЕНО
    log_t = df_log["log_time"].values
    log_p = df_log["log_p_der_smooth"].values  # сглаженные данные
    time_arr = df_log["time"].values

    # 4) Ищем стабильные сегменты методом «скользящего окна»
    segments = segment_log_curve(log_t, log_p, 
                                 min_log_cycle=min_log_cycle,
                                 step_fraction=0.05)
    # segments — список (start_idx, end_idx, k, b)

    # 5) Для каждого сегмента классифицируем наклон
    #    и пытаемся наложить «последовательную» логику (wb→sp→ra/li/bi→pc/im)
    #    Но полностью строгую машинку состояний здесь не реализуем;
    #    просто соберём все сегменты.
    used_modes = set()  # хранить, что уже нашли
    for (s_idx, e_idx, k_seg, b_seg) in segments:
        mode = classify_segment(k_seg, slope_tolerance=slope_tolerance)
        if mode is None:
            continue
        # Сохраняем только первый «ярко выраженный» режим каждого типа
        if results[mode][0] == 0:
            results[mode] = (1, b_seg)

    # 6) Обнаружение граничных эффектов (pc, im) — по «хвосту»
    pc_flag, pc_time, im_flag, im_time = detect_boundaries(log_t, log_p, time_arr)
    if pc_flag == 1:
        results['pc'] = (1, pc_time)
    if im_flag == 1:
        results['im'] = (1, im_time)

    return results


# ============ ПРИМЕР ИСПОЛЬЗОВАНИЯ ============

def process_entire_dataset_advanced(siam_dataset_describe: pd.DataFrame):
    """
    Пример функции, которая идёт по всем файлам из siam_dataset_describe,
    читает их, применяет продвинутую детекцию режимов,
    и возвращает таблицу с результатами.
    """
    results_list = []

    for idx in range(len(siam_dataset_describe)):
        uuid = siam_dataset_describe.iloc[idx, 0]
        # 1) Считываем данные
        df = get_sample(idx, siam_dataset_describe)

        # 2) Обнаруживаем признаки
        regime_info = detect_flow_regimes_advanced(df, 
                                                   min_log_cycle=0.4,
                                                   slope_tolerance=0.25)

        row = {
            'uuid': uuid,
            'bq': regime_info['bq'],
            'wb': regime_info['wb'][0],
            'ra': regime_info['ra'][0],
            'li': regime_info['li'][0],
            'bi': regime_info['bi'][0],
            'sp': regime_info['sp'][0],
            'pc': regime_info['pc'][0],
            'im': regime_info['im'][0],
            # Численные коэффициенты
            'wb_b': regime_info['wb'][1],
            'ra_b': regime_info['ra'][1],
            'li_b': regime_info['li'][1],
            'bi_b': regime_info['bi'][1],
            'sp_b': regime_info['sp'][1],
            'pc_start_time': regime_info['pc'][1],
            'im_start_time': regime_info['im'][1]
        }
        results_list.append(row)

    return pd.DataFrame(results_list)


# ================== DEMO ==================

if __name__ == "__main__":
    # Условный пример DataFrame с именами файлов:

    # Обработка всего набора
    results_df = process_entire_dataset_advanced(frame_test)

    print(results_df)


d:\Python3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
d:\Python3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
d:\Python3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
d:\Python3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
d:\Python3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)
d:\Python3\Lib\site-packages\sklear

                                     uuid  bq  wb  ra  li  bi  sp  pc  im  \
0    e9c7e07f-b723-4da3-918e-7dbcb360c830   0   1   0   1   1   1   0   1   
1    726565c9-e05b-4a28-9079-22d94f9bb9e2   0   0   1   1   1   1   0   0   
2    c2a4d43d-5a93-4c37-81b6-d4fdc5b0280b   0   0   1   0   0   0   0   0   
3    15e04219-9fa0-401d-8b42-833b71ccafb3   0   1   1   1   1   0   0   0   
4    d43f459d-97fa-4ab0-bacc-43e2cb73e1b9   0   0   1   1   1   0   0   0   
..                                    ...  ..  ..  ..  ..  ..  ..  ..  ..   
495  a0448188-be5d-4ad6-929f-1870710e224b   0   1   1   0   1   1   0   0   
496  c295e7be-009c-4893-94c0-194d449077ed   0   0   1   1   1   1   1   0   
497  ac06114e-e466-497a-a0b7-5166d784140e   0   1   1   1   1   0   0   1   
498  0fd9d92c-a6d6-44c2-80b2-7f3cfcbea3da   0   0   1   0   1   0   0   0   
499  b145d37a-2b67-47e1-b4d4-a70a59192187   0   1   0   1   1   1   0   0   

         wb_b      ra_b      li_b      bi_b      sp_b  pc_start_time  \
0  

In [16]:
import numpy as np
import pandas as pd

import math

# pip install ruptures
import ruptures as rpt

# Для сглаживания
from scipy.signal import savgol_filter

# Робастная регрессия
from sklearn.linear_model import RANSACRegressor, LinearRegression

from torch.utils.data import Dataset

# ======= Ваши базовые классы и функции =======

def get_sample(index, frame):
    """
    Чтение файла с данными (time, delta_p, p_) по uuid.
    """
    uuid = frame.iloc[index, 0]
    with open(f"data/{uuid}", "r") as f:
        content = f.read()
    rows = [line.split('\t') for line in content.strip().split('\n')]
    data = pd.DataFrame(rows, columns=['time', 'delta_p', 'p_'], dtype=float)
    return data

class SiamDataset(Dataset):
    def __init__(self, siam_dataset_describe: pd.DataFrame):
        super().__init__()
        self.siam_dataset_describe = siam_dataset_describe

    def __len__(self):
        return self.siam_dataset_describe.shape[0]

    def __getitem__(self, idx):
        df = get_sample(idx, self.siam_dataset_describe)
        # Возвращаем numpy
        return df[["delta_p", "p_"]].to_numpy(dtype=np.float64)

# ======= ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ =======

def check_data_quality(df, min_points=5, max_rel_jump=10.0):
    """
    Эвристика для признака bq.
    Возвращает int: 1 (плохие данные) или 0 (хорошие).
    """
    if len(df) < min_points:
        return 1

    delta_p = df["delta_p"].values
    m = np.mean(delta_p) if np.mean(delta_p) != 0 else 1e-9
    if abs(m) < 1e-12:
        return 1
    diffs = np.abs(np.diff(delta_p))
    if len(diffs) == 0:
        return 1
    rel_jump = np.max(diffs) / abs(m)
    if rel_jump > max_rel_jump:
        return 1
    return 0

def to_log_scale(df):
    """
    Готовим DF с log10(time) и log10(p_).
    Фильтруем время <= 0 и p_ <= 0.
    Сортируем по времени.
    """
    df = df.copy()
    df = df[(df["time"] > 0) & (df["p_"] > 0)].sort_values("time")
    df["log_time"] = np.log10(df["time"])
    df["log_p_der"] = np.log10(df["p_"])
    return df

def savgol_smooth(series, window_size=5, polyorder=2):
    """
    Фильтр Савицкого-Голея (не забудьте, что window_size надо
    подбирать под реальную частоту выборки).
    """
    if len(series) < window_size:
        return series
    return savgol_filter(series, window_length=window_size, polyorder=polyorder)

def robust_regression_log(x_log, y_log):
    """
    Робастная линейная регрессия (RANSAC).
    x_log, y_log - numpy-массивы (1D).
    Возвращает (k, b) или (None, None).
    """
    if len(x_log) < 2:
        return None, None
    
    model = RANSACRegressor(min_samples=max(2, int(len(x_log))),
                            residual_threshold=0.4,  # пример порога, требует настройки
                            max_trials=1)
    X = x_log.reshape(-1, 1)
    try:
        model.fit(X, y_log)
        k = model.estimator_.coef_[0]
        b = model.estimator_.intercept_
        return (k, b)
    except:
        return None, None

def segment_with_ruptures(signal, n_bkps=5):
    """
    Пример сегментации с помощью 'ruptures'.
    signal: 1D numpy-массив (например, мгновенный наклон).
    n_bkps: максимально допустимое число разбиений (эвристика).
    Возвращает список индексов, где заканчиваются сегменты (включая конец массива).
    """
    if len(signal) < 2:
        return [len(signal)]
    
    # Используем, например, Pelt + l2-норму
    model = rpt.Pelt(model="l2").fit(signal)
    # 'pen' — "penalty", нужно подбирать под реальные данные.
    # Или можно использовать 'k' — заданное число изменений.
    # Ниже - пример, где penalty = 2.0
    # Альтернатива: bkps = model.predict(n_bkps=n_bkps)
    bkps = model.predict(pen=2.0)
    return bkps

def classify_slope(k, slope_tolerance=0.1):
    """
    Сопоставляем k с одним из {wb=1, sp=-0.5, ra=0, bi=0.25, li=0.5}.
    Возвращает str или None.
    """
    modes = {
        "wb": 1.0,
        "sp": -0.5,
        "ra": 0.0,
        "bi": 0.25,
        "li": 0.5
    }
    best_mode = None
    best_diff = 999
    for m, kv in modes.items():
        diff = abs(k - kv)
        if diff < best_diff:
            best_diff = diff
            best_mode = m
    if best_diff <= slope_tolerance:
        return best_mode
    else:
        return None

# ======= ГЛАВНАЯ ФУНКЦИЯ =======

def detect_flow_regimes_ultimate(df, min_log_cycle=0.5,
                                 slope_tolerance=0.1,
                                 boundary_fraction=0.25):
    """
    Очень продвинутый подход:
     1) bq
     2) логарифмируем (time, p_)
     3) сглаживаем
     4) вычисляем мгновенный наклон (разностная approx d(log_p)/d(log_t))
     5) сегментируем (ruptures)
     6) на каждом сегменте делаем робастную регрессию и классифицируем
     7) ищем pc/im в хвосте

    Возвращает dict с 8 признаками и их коэффициентами.
    """

    # Результирующая структура
    results = {
        'bq': 0,
        'wb': (0, None),
        'ra': (0, None),
        'li': (0, None),
        'bi': (0, None),
        'sp': (0, None),
        'pc': (0, None),
        'im': (0, None)
    }

    # 1) bq
    results['bq'] = check_data_quality(df)

    # 2) Лог-преобразование
    df_log = to_log_scale(df)
    if len(df_log) < 5:
        return results

    df_log = df_log.reset_index(drop=True)

    # 3) Сглаживаем log_p_der
    smoothed_log_p = savgol_smooth(df_log["log_p_der"].values, window_size=5, polyorder=2)
    df_log["log_p_smooth"] = smoothed_log_p# df_log["log_p_der"] #smoothed_log_p

    log_t = df_log["log_time"].values
    log_p = df_log["log_p_smooth"].values
    times = df_log["time"].values

    # 4) Вычисляем «мгновенный наклон» k_i ~ d(log_p)/d(log_t)
    #    Для простоты возьмём центральную разность.
    slopes = np.zeros_like(log_p)
    for i in range(1, len(log_p)-1):
        dy = log_p[i+1] - log_p[i-1]
        dx = log_t[i+1] - log_t[i-1]
        slopes[i] = dy / dx if abs(dx) > 1e-12 else 0
    slopes[0] = slopes[1]  # граничные ячейки как-то экстраполируем
    slopes[-1] = slopes[-2]

    # 5) Сегментация массива slopes
    #    n_bkps - максимально допустимое количество «разрывов».
    #    Если у вас обычно до 3-5 смен режима, можно ставить 5-7.
    #    Игра с penalty и n_bkps — ключ к точной сегментации.
    bkps = segment_with_ruptures(slopes, n_bkps=1)  # список индексов конца сегментов

    # 6) Проходим по сегментам, делаем робастную регрессию (log_t, log_p) и классифицируем
    #    Также проверяем, что сегмент не короче 0.5 лог-цикла
    start_idx = 0
    found_modes = []  # [(mode, start_idx, end_idx, k, b), ...]
    for bkp in bkps:
        end_idx = bkp
        if end_idx - start_idx < 2:
            start_idx = end_idx
            continue
        # Достаём данные сегмента
        seg_log_t = log_t[start_idx:end_idx]
        seg_log_p = log_p[start_idx:end_idx]
        seg_times = times[start_idx:end_idx]

        if len(seg_log_t) < 2:
            start_idx = end_idx
            continue

        # Проверяем длину сегмента в логарифмическом масштабе
        t_min, t_max = seg_times[0], seg_times[-1]
        if t_min <= 0 or t_max <= 0:
            # защита от дурака
            start_idx = end_idx
            continue
        log_length = math.log10(t_max) - math.log10(t_min)
        if log_length < min_log_cycle:
            # Слишком короткий, возможно не настоящий режим
            start_idx = end_idx
            continue

        # Робастная регрессия
        k_seg, b_seg = robust_regression_log(seg_log_t, seg_log_p)
        if k_seg is None:
            start_idx = end_idx
            continue

        # Классифицируем (wb, sp, ra, li, bi или None)
        mode = classify_slope(k_seg, slope_tolerance=slope_tolerance)
        if mode is not None:
            found_modes.append((mode, start_idx, end_idx, k_seg, b_seg))

        start_idx = end_idx

    # Сохраняем **все** найденные режимы, если хотим (в задаче обычно нужен первый яркий).
    # Для демонстрации сохраним только первый «сильный» сегмент каждого типа:
    for (mode, s_idx, e_idx, k_, b_) in found_modes:
        if results[mode][0] == 0:
            # ещё не зафиксировали такой режим
            results[mode] = (1, b_)

    # 7) Определяем pc/im в хвосте
    #    Можно взять последние boundary_fraction процентов точек
    tail_start = int((1-boundary_fraction)*len(df_log))
    if tail_start < 0:
        tail_start = 0

    tail_log_t = log_t[tail_start:]
    tail_log_p = log_p[tail_start:]
    tail_times = times[tail_start:]

    k_tail, b_tail = robust_regression_log(tail_log_t, tail_log_p)
    if k_tail is not None and len(tail_times) > 0:
        t_boundary = tail_times[0]  # момент начала последнего сегмента
        # эвристика:
        if k_tail < -0.2:
            results['pc'] = (1, t_boundary)
        elif k_tail > 0.7:
            results['im'] = (1, t_boundary)

    return results


# ======= Пример функции, обрабатывающей весь DataFrame =======

def process_entire_dataset_ultimate(siam_dataset_describe: pd.DataFrame):
    """
    Проходим по всем файлам, читаем их, определяем признаки расширенным методом.
    """
    results_list = []
    for idx in range(len(siam_dataset_describe)):
        uuid = siam_dataset_describe.iloc[idx, 0]
        df = get_sample(idx, siam_dataset_describe)

        regime_info = detect_flow_regimes_ultimate(
            df,
            min_log_cycle=0.3,
            slope_tolerance=0.2,
            boundary_fraction=0.25
        )

        row = {
            'uuid': uuid,
            'bq': regime_info['bq'],
            'wb': regime_info['wb'][0],
            'ra': regime_info['ra'][0],
            'li': regime_info['li'][0],
            'bi': regime_info['bi'][0],
            'sp': regime_info['sp'][0],
            'pc': regime_info['pc'][0],
            'im': regime_info['im'][0],
            # Численные коэффициенты
            'wb_b': regime_info['wb'][1],
            'ra_b': regime_info['ra'][1],
            'li_b': regime_info['li'][1],
            'bi_b': regime_info['bi'][1],
            'sp_b': regime_info['sp'][1],
            'pc_start_time': regime_info['pc'][1],
            'im_start_time': regime_info['im'][1]
        }
        results_list.append(row)
    return pd.DataFrame(results_list)


if __name__ == "__main__":
    # Пример DataFrame со списком uuid
    # Обрабатываем
    res = process_entire_dataset_ultimate(frame_test)
    print(res)


d:\Python3\Lib\site-packages\sklearn\metrics\_regression.py:1266: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


                                     uuid  bq  wb  ra  li  bi  sp  pc  im  \
0    e9c7e07f-b723-4da3-918e-7dbcb360c830   0   0   1   1   0   0   0   1   
1    726565c9-e05b-4a28-9079-22d94f9bb9e2   0   0   1   0   1   0   0   0   
2    c2a4d43d-5a93-4c37-81b6-d4fdc5b0280b   0   0   0   0   0   0   0   1   
3    15e04219-9fa0-401d-8b42-833b71ccafb3   0   0   1   0   0   0   0   0   
4    d43f459d-97fa-4ab0-bacc-43e2cb73e1b9   0   0   1   1   1   0   0   0   
..                                    ...  ..  ..  ..  ..  ..  ..  ..  ..   
495  a0448188-be5d-4ad6-929f-1870710e224b   0   1   0   0   0   0   1   0   
496  c295e7be-009c-4893-94c0-194d449077ed   0   0   1   0   0   0   1   0   
497  ac06114e-e466-497a-a0b7-5166d784140e   0   0   1   1   0   0   0   1   
498  0fd9d92c-a6d6-44c2-80b2-7f3cfcbea3da   0   0   1   0   0   0   1   0   
499  b145d37a-2b67-47e1-b4d4-a70a59192187   0   0   0   0   1   0   0   0   

        wb_b      ra_b      li_b      bi_b  sp_b  pc_start_time  im_start_t